# Demucs Music Source Separation
reference: https://github.com/facebookresearch/demucs

In [16]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-x5pzdgn8/demucs_e0f4751428f3499e998ff6a5d921fe38
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-x5pzdgn8/demucs_e0f4751428f3499e998ff6a5d921fe38
  Resolved https://github.com/facebookresearch/demucs to commit 5d2ccf224f595b8654b0447e06f6adc866cca61a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
ls

 audio/  'Moves Like Jagger (Maroon 5) fingerstyle guitar.mp4'
 code/    期末報告.pptx


In [19]:
import os
os.chdir('/content/drive/Shareddrives/music_folder')

In [20]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance two_stems = "vocals"

python_env = "python3"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = './audio/band/chorus/original'
out_path = './audio/band/chorus/demucs_separated'

In [21]:
# Useful functions, don't forget to execute
import io
from pathlib import Path
import select
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

#return a list of files that are our target extentions
def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

#get command for separation
def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = [python_env, "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")

In [22]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
separate()

Going to separate the files:
audio/band/chorus/original/Adele - Skyfall_chorus.mp3
audio/band/chorus/original/John Denver - Take Me Home, Country Roads_chorus.mp3
audio/band/chorus/original/Coldplay - Yellow_chorus.mp3
audio/band/chorus/original/John Lennon - Imagine_chorus.mp3
audio/band/chorus/original/Moves Like Jagger - Maroon 5_chorus.mp3
audio/band/chorus/original/Always - Bon Jovi_chorus.mp3
audio/band/chorus/original/Jack Johnson - Better Together_chorus.mp3
audio/band/chorus/original/Coldplay - Hymn For The Weekend _chorus.mp3
audio/band/chorus/original/John Legend - All of Me _chorus.mp3
With command:  python3 -m demucs.separate -o ./audio/band/chorus/demucs_separated -n htdemucs --mp3 --mp3-bitrate=320


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:01<00:00, 79.6MB/s]


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/drive/Shareddrives/music_folder/audio/band/chorus/demucs_separated/htdemucs
Separating track audio/band/chorus/original/Adele - Skyfall_chorus.mp3


100%|██████████████████████████████████████████████| 17.549999999999997/17.549999999999997 [00:41<00:00,  2.34s/seconds]


Separating track audio/band/chorus/original/John Denver - Take Me Home, Country Roads_chorus.mp3


...

100%|██████████████████████████████████████████████| 17.549999999999997/17.549999999999997 [00:38<00:00,  2.18s/seconds]


Separating track audio/band/chorus/original/Coldplay - Hymn For The Weekend _chorus.mp3


100%|██████████████████████████████████████████████| 17.549999999999997/17.549999999999997 [00:38<00:00,  2.21s/seconds]


Separating track audio/band/chorus/original/John Legend - All of Me _chorus.mp3


100%|██████████████████████████████████████████████| 17.549999999999997/17.549999999999997 [00:41<00:00,  2.36s/seconds]


# Open-Unmix - A Reference Implementation for Music Source Separation
reference: https://github.com/sigsep/open-unmix-pytorch

In [ ]:
!python3 -m pip install openunmix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/Shareddrives/music_folder')

In [ ]:
ls

audio/  code/


In [31]:
!umx "./audio/band/chorus/original/John Lennon - Imagine_chorus.mp3" --model umxl

  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/openunmix/utils.py:298: UserWarning: resample to model sample rate
  warnings.warn("resample to model sample rate")
/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:1458: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  warnings.warn(
100% 1/1 [01:00<00:00, 60.28s/it]
